Vamos a ver como hacer un web scraping con selenium

Lo primero que tenemos que hacer es activar el manejador para eso tenemos que tener instalado el chromedriver

In [48]:
import pandas as pd
from selenium import webdriver


In [2]:
/html/body/div[5]/table/tbody/tr/td[2]/div[1]/div[4]/div[2]/div[1]/div[2]/div[3]

NameError: name 'html' is not defined

In [3]:
chrome_driver_path = r'C:\\Users\\artur\\Desktop\\repo_arturo\\Chrome driver web scrapping\\chromedriver.exe'

options = webdriver.ChromeOptions()
#options.add_argument('headless')

driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)

El objeto driver es con el que trabajaremos a partir de ahora aqui le indicaremos que pagina queremos scrapear

In [4]:
url = 'https://www.filmaffinity.com/es/main.html'
driver.get(url)

Nos hemos encontrado con un pop-up que nos queremos saltar.
- Buscamos el botón
- Hacemos click en el botón

selenium nos permite buscar elementos por etiqueteas de html: https://selenium-python.readthedocs.io/locating-elements.html

CUIDADO! Los metodos de busqueda de elementos estan "duplicados". Tenemos *find_element_by_tag_name* y *find_elements_by_tag_name* el primero nos devolverá el primer elemento que encuentre (aunque haya más), el segundo nos devolverá una lista con todos los elementos que encuentre (aunque solo sea uno)

In [5]:
element_by_tag = driver.find_elements_by_tag_name('button')
element_by_class_name = driver.find_elements_by_class_name('css-flk0bs')
element_by_xpath = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/div/button[2]')

Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [6]:
dir(element_by_xpath)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 '_w3c',
 'clear',
 'click',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_name',
 'find_element_by_partial_link_text',
 'find_element_by_tag_name',
 'find_element_by_xpath',
 'find_elements',
 'find_elements_by_class_name',
 'find_elements_by_css_selector',
 'find_elements_by_id',
 'find_elements_by_link_text',
 'find_elements_by_name',
 'find_elements_by_partial_link_text',
 'find_elements_by_tag_name',
 'find_elements_by_xpath',
 'get_attribute',
 'get_property',
 'id',
 

In [7]:
element_by_xpath.get_attribute('size')

'large'

Podemos evaluar que tipo de elemento es (tag)

In [8]:
element_by_xpath.tag_name

'button'

Podemos sacar el valor que tiene (el texto)

In [9]:
element_by_xpath.text

'ACEPTO'

Incluso podemos guardar una imagen del elemento

In [10]:
element_by_xpath.screenshot('acepto.png')

True

Evaluemos que elementos hemos encontrado por el tag

In [11]:
for index, element in enumerate(element_by_tag):
    print('Elemento: ', index)
    print('Este elemento tiene este texto: ', element.text)
    print('Este elemento tiene este tag: ', element.tag_name)

    element.screenshot(element.text + '.png') 

Elemento:  0
Este elemento tiene este texto:  socios
Este elemento tiene este tag:  button
Elemento:  1
Este elemento tiene este texto:  MÁS OPCIONES
Este elemento tiene este tag:  button
Elemento:  2
Este elemento tiene este texto:  ACEPTO
Este elemento tiene este tag:  button


Nos quedamos con el boton que nos interesa

In [12]:
boton_aceptar = element_by_tag[2]

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Como hacer click

In [13]:
boton_aceptar.click()

Buscamos una pelicula

In [14]:
from selenium.webdriver.common.keys import Keys

In [15]:
buscador = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[2]/form/input[1]')
buscador.send_keys('El gran lebowski')
buscador.send_keys(Keys.ENTER)

In [16]:
driver.back()

Vamos a buscar todas las peliculas que se estrenan el proximos viernes

1. Nos cogemos los containers que hay en la zona central

In [17]:
menu_lateral = driver.find_element_by_id('lsmenu')
menu_lateral

<selenium.webdriver.remote.webelement.WebElement (session="573749b959a81ed6c29cd8fcafd7d747", element="484926fa-419c-4a40-9d84-d538d72dfd1c")>

In [18]:
mis_as = menu_lateral.find_elements_by_tag_name('a')
for una_a in mis_as:
    if una_a.text == 'Próximos estrenos':
        una_a.click()
        break

2. Vemos con cual nos tenemos que quedar

In [20]:
semanas = driver.find_elements_by_id('main-wrapper-rdcat')

In [26]:
for semana in semanas:
    #print(semana.find_element_by_tag_name('div').text)
    fecha = semana.find_element_by_tag_name('div').get_attribute('id') # OTRA FORMA DE OBTENER LAS SEMANAS
    if fecha == '2021-03-05':
        break

3. Dentro del container tenemos que ir al boton de "ver más"

In [37]:
caratulas = semana.find_elements_by_class_name('mc-poster')
lista_links = []
for peli in caratulas:
    lista_links.append(peli.find_element_by_tag_name('a').get_attribute('href'))

lista_links
    

['https://www.filmaffinity.com/es/film145565.html',
 'https://www.filmaffinity.com/es/film264807.html',
 'https://www.filmaffinity.com/es/film918998.html',
 'https://www.filmaffinity.com/es/film874262.html',
 'https://www.filmaffinity.com/es/film563001.html',
 'https://www.filmaffinity.com/es/film182307.html']

4. Una vez en el div del boton tenemos que llegar a la *a* 

In [39]:
driver.get(lista_links[2])

5. Ahora podríamos hacer click como hemos hecho antes con el boton de aceptar las cooks

In [40]:
titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

In [42]:
print(titulo)
print(nota)
print(votos)
print(ficha)

Y llovieron pájaros
5,8
109
<selenium.webdriver.remote.webelement.WebElement (session="573749b959a81ed6c29cd8fcafd7d747", element="bdea601d-12a1-41d9-877c-b592736717ab")>


Accedemos al container de la semana que queremos ver, exactamente igual que hemos hecho antes

In [44]:
ficha_names = []
ficha_values = []

for name in ficha.find_elements_by_tag_name('dt'):
    ficha_names.append(name.text)

for value in ficha.find_elements_by_tag_name('dd'):
    ficha_values.append(value.text)

ficha_values

['Il pleuvait des oiseaux aka',
 '',
 '2019',
 '124 min.',
 ' Canadá',
 'Louise Archambault',
 'Louise Archambault (Novela: Jocelyne Saucier)',
 'David Ratté, Andréa Bélanger',
 'Mathieu Laverdière',
 'Kenneth Welsh, Andrée Lachapelle, Gilbert Sicotte, Rémy Girard, Ève Landry, Éric Robidoux, Louise Portal',
 'Films Outsiders',
 'Drama | Vejez/Madurez',
 'Esta es la historia de tres ancianos que han elegido retirarse del mundo y vivir en los bosques de Canadá. Al tiempo que un gran incendio amenaza la región, alguien llega hasta su escondite: una joven fotógrafa que busca a un tal Boychuck. Y no es la única. Poco después, una mujer de más de 80 años aparece como una brisa ligera que alborotará sus vidas. Mientras intentan comprender la historia de Boychuck a través de sus pinturas, algo extraordinario surgirá entre todos ellos. (FILMAFFINITY)']

In [49]:
columnas = ['Titulo', 'Nota', 'Votos']
columnas.extend(ficha_names)
columnas

values = [titulo, nota, votos]
values.extend(ficha_values)
values

pd.DataFrame([values], columns=columnas)

,Titulo,Nota,Votos,Título original,,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Sinopsis
0,Y llovieron pájaros,"5,8",109,Il pleuvait des oiseaux aka,,2019,124 min.,Canadá,Louise Archambault,Louise Archambault (Novela: Jocelyne Saucier),"David Ratté, Andréa Bélanger",Mathieu Laverdière,"Kenneth Welsh, Andrée Lachapelle, Gilbert Sico...",Films Outsiders,Drama | Vejez/Madurez,Esta es la historia de tres ancianos que han e...


Buscamos las peliculas

['https://www.filmaffinity.com/es/film783812.html',
 'https://www.filmaffinity.com/es/film766252.html',
 'https://www.filmaffinity.com/es/film228161.html',
 'https://www.filmaffinity.com/es/film244293.html',
 'https://www.filmaffinity.com/es/film350880.html',
 'https://www.filmaffinity.com/es/film674272.html',
 'https://www.filmaffinity.com/es/film687348.html',
 'https://www.filmaffinity.com/es/film838012.html',
 'https://www.filmaffinity.com/es/film561634.html']

Vamos a crear una función que nos haga todo esto

## Modo Dios moviendonos entre ventanas

Vamos a ver como nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [54]:
driver.execute_script('window.open("");')

Movernos a otra ventana

In [60]:
driver.switch_to.window(driver.window_handles[1])

Cerrar ventana

In [59]:
driver.close()

Una vez cerramos la ventana tenemos que indicarle a que ventana tiene que ir

In [73]:
driver.switch_to.window(driver.window_handles[-1])

Sabiendo como podemos movernos por entre las ventanas y sabiendo como extraer de cada pagina toda la información que necesitamos vamos a crear nuestro dataframe

In [62]:

for link in lista_links:
    driver.execute_script('window.open("");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)

In [72]:
# FUNCIONES PARA CREAR EL DATAFRAME CON TODAS LAS PELICULAS

def sacar_info(driver):

    titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text

    try:
        nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
        votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
    
    except:

        nota = None
        votos = None
        
    ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

    return titulo, nota, votos, ficha

def ficha_tecnica(ficha):

    ficha_names = []
    ficha_values = []

    for name in ficha.find_elements_by_tag_name('dt'):
        ficha_names.append(name.text)

    for value in ficha.find_elements_by_tag_name('dd'):
        ficha_values.append(value.text)

    return ficha_names, ficha_values

def contruir_df(ficha_names, ficha_values, titulo, nota, votos):

    columnas = ['Titulo', 'Nota', 'Votos']
    columnas.extend(ficha_names)
    

    values = [titulo, nota, votos]
    values.extend(ficha_values)
    
    return pd.DataFrame([values], columns=columnas)

def nueva_pelicula(driver):

    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_names, ficha_values = ficha_tecnica(ficha)
    mi_peli = contruir_df(ficha_names, ficha_values, titulo, nota, votos)

    return mi_peli

In [74]:
peliculas = pd.DataFrame()

for link in lista_links:
    driver.execute_script('window.open("");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)

    nueva_peli = nueva_pelicula(driver)
    peliculas = peliculas.append(nueva_peli)

peliculas

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Grupos,Sinopsis,
0,Raya y el último dragón,None,None,Raya and the Last Dragon,2021,114 min.,Estados Unidos,"Don Hall, Carlos López Estrada, Paul Briggs, J...","Qui Nguyen, Adele Lim (Historia: Paul Briggs, ...",James Newton Howard,Animación,Animación,Walt Disney Animation Studios (Distribuidora: ...,Animación. Fantástico. Aventuras | Dragones,Clásicos de Walt Disney,"En el fantástico mundo de Kumandra, humanos y ...",NaN
0,The Owners (Los propietarios),"4,9",436,The Owners,2020,92 min.,Reino Unido,Julius Berg,"Julius Berg, Matthieu Gompel, Geoff Cox (Cómic...","Jean Paul Frazer, Vincent Welch",David Ungaro,"Maisie Williams, Rita Tushingham, Sylvester Mc...","Co-production Reino Unido-Canadá; Blue Light, ...",Terror. Thriller | Años 90. Thriller psicológico,NaN,"Inglaterra rural, principios de los noventa: l...",NaN
0,Y llovieron pájaros,"5,8",109,Il pleuvait des oiseaux aka,2019,124 min.,Canadá,Louise Archambault,Louise Archambault (Novela: Jocelyne Saucier),"David Ratté, Andréa Bélanger",Mathieu Laverdière,"Kenneth Welsh, Andrée Lachapelle, Gilbert Sico...",Films Outsiders,Drama | Vejez/Madurez,NaN,Esta es la historia de tres ancianos que han e...,
0,Birdsong,"5,6",21,Birdsong,2019,90 min.,Bélgica,Hendrik Willemyns,Hendrik Willemyns,"Timothy Bruzon, Hendrik Willemyns",Michael Dwyer,"Kazuhiko Kanayama, Natsuko Kobayashi, Akaji Ma...",Co-production Bélgica-Japón; Menuetto Film,Drama. Fantástico | Música,NaN,"Tokio. Asuka, una joven de la limpieza que tra...",NaN
0,Woman,"7,4",21,Woman,2019,105 min.,Francia,"Yann Arthus-Bertrand, Anastasia Mikova","Anastasia Mikova, Yann Arthus-Bertrand",NaN,NaN,Documental,Hope Production,Documental | Feminismo,NaN,Documental que nos lleva por todos los rincone...,NaN
0,La Mami,"6,4",298,La Mami,2019,80 min.,México,Laura Herrero Garvín,NaN,NaN,Laura Herrero Garvín,Documental,"Co-production México-España; Cacerola Films, G...",Documental | Feminismo. Basado en hechos reale...,NaN,"En el cabaret Barba Azul, en Ciudad de México,...",NaN
